# DIRECT core analysis
**nick budak // @thatbudakguy**

The DIRECT `dphon` tool works by comparing two utf-8 encoded plaintext files (hereafter referred to as `a` and `b` in the general case) and outputting a list of matching sequences, based on a phonological dictionary that corresponds to the Schuessler 2007 reconstruction of Old Chinese[^1].

There are four main steps to this process, each of which is currently implemented as a method on the `Comparator` object exported in `dphon.lib`:

|step|name|method|input|output|
|---|---|---|---|---|
|1|shingle|`Comparator.get_text_ngrams`|`a` or `b`, as string|list of ngrams in `a` or `b`|
|2|match|`Comparator.get_initial_matches`|lists of ngrams from `a` and `b`|list of `Match` objects|
|3|reduce|`Comparator.reduce_matches`|list of `Match` objects|list of `Match` objects|
|4|group|`Comparator.group_matches`|list of `Match` objects|dict with `Match` keys and `Match` values|

After this process is finished we will have an organized list of matches between `a` and `b`, ordered by their occurrence in `a`. The `Match` object itself contains no character data - only the positions at which it occurs in the source text. Thus, any presentation to the user will involve calling a final method `Comparator.resolve_groups` to apply the `Match` objects to `a` and `b` and receive strings to format and print to the console.

[^1]: Schuessler, Axel (2007), ABC Etymological Dictionary of Old Chinese, Honolulu: University of Hawaii Press, ISBN 978-0-8248-2975-9.

## shingling
the first step is to break our two texts into a series of overlapping n-grams, a process called shingling. while other language models may do this at the level of words, chinese is not a whitespace-delimited language, and so DIRECT does it at the level of individual characters. the default value for `n` is 3, so we will be shingling our two inputs texts into lists of overlapping trigrams.


In [2]:
from dphon.lib import Comparator

a = 'AABDCB'
b = 'AABDDA'

ab_dict = {
    'A': [None, None, '1'],
    'B': [None, None, '2'],
    'C': [None, None, '3'],
}

c = Comparator(phondict=ab_dict)

a_grams = c.get_text_ngrams(a)
b_grams = c.get_text_ngrams(b)

print('n-grams in a:')
print([g['text'] for g in a_grams])
print('\n')
print('n-grams in b:')
print([g['text'] for g in b_grams])

n-grams in a:
['112', '12D', '2D3', 'D32']


n-grams in b:
['112', '12D', '2DD', 'DD1']


## matching
to get our initial set of matches, we need to compare each n-gram from `a` against every n-gram in `b` and record when the two are identical. this ensures that matches that might involve the same content but occur in different places are preserved, since they would otherwise all be grouped together. this step will create a list of `Match` objects, which store the start and end positions in the original versions of each of the two texts. we can get a printable representation of a `Match` by calling `resolve()` and passing in the two texts whose positions are stored in the `Match`.

In [3]:
matches = c.get_initial_matches(a_grams, b_grams)

print('initial matches:')
print('----------------')
for m in matches:
    print('%s\t%s' % (m.resolve(a, b), m))

initial matches:
----------------
AAB :: AAB	A (0 - 2) :: B (0 - 2)
ABD :: ABD	A (1 - 3) :: B (1 - 3)


## reducing
In order to capture the 4-character match at the start of `a` and `b` in its entirety, we'll need to iteratively build up these trigrams into the longest possible matching sequences. This step illustrates a property of text n-grams: any single sequence of length n can be decomposed into two overlapping sequences of length n-1. Thus we can decompose any arbitrarily long match into some number of overlapping trigrams, and conversely we can "rebuild" a match of any length by combining the trigrams we got in step 1.

In [4]:
matches = c.reduce_matches(matches)

print('reduced matches:')
print('----------------')
for m in matches:
    print('%s\t%s' % (m.resolve(a, b), m))

reduced matches:
----------------
AABD :: AABD	A (0 - 3) :: B (0 - 3)


## grouping

we now have a list of matching sequences between `a` and `b` optimized for length, but our output could be more succinct. if a sequence in `a` matches multiple locations in `b`, we end up repeating a lot of redundant information. grouping lets us collect all the sequences in `b` matching our sequence in `a` under a single heading, making the output easier to browse and interpret.


In [5]:
groups = c.group_matches(matches)

print('grouped matches:')
print('----------------')
for ga, gbs in groups.items():
    print('%s: A (%d-%d)' % (a[ga.start:ga.stop+1], ga.start, ga.stop))
    for gb in gbs:
        print('%s: B (%d-%d)\n' % (b[gb.start:gb.stop+1], gb.start, gb.stop))
    print('\n')

print('resolved groups:')
print('----------------')
print(c.resolve_groups(a, b, groups))

grouped matches:
----------------
AABD: A (0-3)
AABD: B (0-3)



resolved groups:
----------------
AABD (a: 1)
AABD (b: 1)


